In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D,MaxPooling1D





In [5]:
movies = pd.read_csv("../datasets/reseñas_limpio.csv")


In [3]:
y = movies['sentiment'] #NUESTRA VARIABLE Y CON 0 Y 1 PARA LAS ETIQUETAS DEL SENTIMIENTO

y = np.array(list(map(lambda x: 1 if x=="positive" else 0, y))) #CON ESTO LO QUE HAGO ES ESTABLECER EN 1 LOS POSITIVOS 
#Y EN 0 LOS NEGATIVOS.

X = movies["review"] #LA VARABLE X SERAN LAS LISTAS DE PALABRAS DE RESEÑAS

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
#ESTABLEZCO UNA PROPORCION 0.20 PARA EL CONJUNTO DE PRUEBA Y 0.80 PARA EL DE ENTRENAMIENTO.

In [5]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)#compruebo que todas las dimensiones están bien.

(40000,) (10000,) (40000,) (10000,)


In [6]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)


In [7]:
token = Tokenizer(num_words=5000)  #establezco el número máximo de palabras que se pueden conservar, según la frecuencia de palabras.
token.fit_on_texts(X_train) #actualizo el vocabulario en el x train

X_train = token.texts_to_sequences(X_train) #transformamos nuestros tokens en numeros para x train y para x test
X_test = token.texts_to_sequences(X_test)

In [8]:

vocab_size = len(token.word_index) + 1 #Todas las palabras y sus índices se almacenarán en un diccionario al que podre acceder 



X_train = pad_sequences(X_train, padding='post', maxlen=100) #dejo un maximo de 100 palabras por lista.
X_test = pad_sequences(X_test, padding='post', maxlen=100)
vocab_size #nuestro corpus tiene 91098 palabras únicas.

91098

In [9]:
X_train.shape #ahora tenemos 40000 listas con 100 palabras

(40000, 100)

In [10]:
X_test.shape #igual que en el test.

(10000, 100)

In [25]:
model = Sequential()
embedding_layer = Embedding(5000,32, input_length=100 , trainable=False)
model.add(embedding_layer)
model.add(Conv1D(128,5,activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 100, 32)           160000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 96, 128)           20608     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 48, 128)           0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 6144)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 250)               1536250   
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 251       
Total params: 1,717,109
Trainable params: 1,557,109
Non-trainable params: 160,000
______________________________________

In [29]:
model.fit(X_train,y_train, validation_data=(X_test,y_test),epochs = 2, batch_size=32,verbose=1)
scores = model.evaluate(X_test,y_test,verbose=1)
print("Precisión: %.2f%%" % (scores[1]*100))


Epoch 1/2
1250/1250 [==============================] - 50s 40ms/step - loss: 0.3043 - accuracy: 0.8701 - val_loss: 0.5560 - val_accuracy: 0.7483
Epoch 2/2
313/313 [==============================] - 3s 10ms/step - loss: 0.6331 - accuracy: 0.7446
Precisión: 74.46%
